In [1]:
import numpy as np
import pandas as pd

In [82]:
raw_data=pd.read_csv("../data/cleaned_data.csv")
tendecy_one_hot=pd.get_dummies(raw_data.loc[:,"Tendency"])
raw_data=pd.concat([raw_data,tendecy_one_hot],axis=1)
raw_data.drop(['Tendency','DR'],axis=1, inplace=True)
raw_data.columns

Index(['FileName', 'Date', 'SegFile', 'b', 'e', 'LBE', 'LB', 'AC', 'FM', 'UC',
       'ASTV', 'MSTV', 'ALTV', 'MLTV', 'DL', 'DS', 'DP', 'Width', 'Min', 'Max',
       'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'NSP',
       'left_assymetric', 'right_symmetric', 'symmetric'],
      dtype='object')

In [108]:
continuous_attributes=raw_data.iloc[:,3:26]
categorical_attributes=raw_data.iloc[:,26:]
continuous_attributes=(continuous_attributes-continuous_attributes.min())/(continuous_attributes.max()-continuous_attributes.min())
dataset=pd.concat([continuous_attributes,categorical_attributes.iloc[:,1:],categorical_attributes.iloc[:,0]],axis=1)
dataset.to_csv("../data/cleaned_normalized_data.csv",index=False)

In [124]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

In [125]:
dataset_array=dataset.values
data_x=dataset_array[:,:-1]
data_y=dataset_array[:,-1]
train_x=dataset_array[:1200,:-1]
train_y=dataset_array[:1200,-1]
test_x=dataset_array[1200:,:-1]
test_y=dataset_array[1200:,-1]

In [143]:
classifier = svm.SVC(kernel='rbf', C = 1.0, verbose=True)

skf=StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(data_x, data_y):
    X_train, X_test = data_x[train_index], data_x[test_index]
    y_train, y_test = data_y[train_index], data_y[test_index]
    classifier.fit(X_train,y_train)
    y_pred=classifier.predict(X_test)
    confusion_mtx = confusion_matrix(y_test, y_pred) 
    print("\n",confusion_mtx)
    target_names = ['normal', 'suspect', 'pathological']
    print(classification_report(y_test, y_pred, target_names=target_names))
    print("accuracy= ",(confusion_mtx.diagonal().sum()/confusion_mtx.sum())*100)



[LibSVM]
 [[315  16   0]
 [ 31  28   0]
 [  6  26   4]]
              precision    recall  f1-score   support

      normal       0.89      0.95      0.92       331
     suspect       0.40      0.47      0.43        59
pathological       1.00      0.11      0.20        36

 avg / total       0.84      0.81      0.79       426

accuracy=  81.455399061
[LibSVM]
 [[321  10   0]
 [ 26  33   0]
 [ 11  20   4]]
              precision    recall  f1-score   support

      normal       0.90      0.97      0.93       331
     suspect       0.52      0.56      0.54        59
pathological       1.00      0.11      0.21        35

 avg / total       0.85      0.84      0.82       425

accuracy=  84.2352941176
[LibSVM]
 [[331   0   0]
 [ 43  16   0]
 [  7  10  18]]
              precision    recall  f1-score   support

      normal       0.87      1.00      0.93       331
     suspect       0.62      0.27      0.38        59
pathological       1.00      0.51      0.68        35

 avg / total       